This notebook is going to focus on processing the players' season statistics only. The other three files will be processed in separate notebooks.

In [1]:
# Importing standard packages for data exploration and processing.
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)


# Unlike in the Stage 1 notebooks, we are going to create new variables rather than perform the operations in-place here.
# The reason is that we might need to review the original data during processing.
raw_players_season = pd.read_csv('../raw_data/raw_players_season.csv')

# Does everything seem to be alright with the data?
raw_players_season.head()

,URL,Player name,Season,Tournament / Team,№,GP,G,Assists,PTS,+/-,+,-,PIM,ESG,PPG,SHG,OTG,GWG,SDS,SOG,%SOG,S/G,FO,FOW,%FO,TOI/G,SFT/G,HITS,BLS,FOA,W,L,SOP,GA,Sv,%Sv,GAA,SO,TOI
0,https://en.khl.ru/players/16673/,Sergei Abramov,Regular season 2014/2015,Amur (Khabarovsk),93.0,13.0,1.0,0.0,1.0,-4.0,1.0,5.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,9.1,0.8,0.0,0.0,-,6:57,9.3,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://en.khl.ru/players/16673/,Sergei Abramov,Nadezhda Cup 2013/2014,Amur (Khabarovsk),91.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,0.0,0.0,-,2:44,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.khl.ru/players/16673/,Sergei Abramov,Regular season 2013/2014,Amur (Khabarovsk),91.0,12.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,1.2,1.0,0.0,0.0,6:15,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.khl.ru/players/16673/,Sergei Abramov,Nadezhda Cup 2012/2013,Amur (Khabarovsk),99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,0.0,0.0,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.khl.ru/players/16462/,Maxim Alyapkin,Regular season 2015/2016,Torpedo (Nizhny Novgorod Region),31.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,3.0,10.0,76.9,2.98,0.0,60:25


We can already see that there are some issues with missing data and integers stored as floats.

In [2]:
# What would the summary tell us?
raw_players_season.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19674 entries, 0 to 19673
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   URL                19674 non-null  object 
 1   Player name        19674 non-null  object 
 2   Season             19674 non-null  object 
 3   Tournament / Team  19674 non-null  object 
 4   №                  18677 non-null  float64
 5   GP                 19674 non-null  float64
 6   G                  19674 non-null  float64
 7   Assists            19674 non-null  float64
 8   PTS                17753 non-null  float64
 9   +/-                17753 non-null  float64
 10  +                  17753 non-null  float64
 11  -                  17753 non-null  float64
 12  PIM                19674 non-null  float64
 13  ESG                17753 non-null  float64
 14  PPG                17753 non-null  float64
 15  SHG                17753 non-null  float64
 16  OTG                177

We can see that in many columns there is no missing data at all. At the same time, for other columns there is a clear separation into skaters (forwards and defencemen) and goalies.

For example, we can see that season statistics appears to have 7859 rows of data for skaters and 1113 rows for goalies, with a total of 8972 rows. However, there are 8974 rows in the dataframe so 2 rows seem to be unaccounted in either.

Let us find out who is messing up our data. Icetime seems like a good indicator since it must be present for all players who have recorded a match during that season and is stored differently for skaters (average icetime per match) and goalies (total icetime per season).

In [3]:
# We need the rows for which both icetime are null.
raw_players_season[raw_players_season['TOI/G'].isnull() & raw_players_season['TOI'].isnull()]

,URL,Player name,Season,Tournament / Team,№,GP,G,Assists,PTS,+/-,+,-,PIM,ESG,PPG,SHG,OTG,GWG,SDS,SOG,%SOG,S/G,FO,FOW,%FO,TOI/G,SFT/G,HITS,BLS,FOA,W,L,SOP,GA,Sv,%Sv,GAA,SO,TOI
2037,https://en.khl.ru/players/29144/,David Boldizar,Regular season 2018/2019,Slovan (Bratislava),61.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2038,https://en.khl.ru/players/29144/,David Boldizar,Regular season 2017/2018,Slovan (Bratislava),23.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So, David Boldizar from Slovan (Bratislava) is the culprit. I wonder what is going on with him. Thankfully, we have added each player's profile link so we can easily check the original data, and it turns out that the data was not stored properly on the website to begin with.

In [4]:
# What do we know about that specific player?
raw_players_season[raw_players_season['URL'] == 'https://en.khl.ru/players/29144/'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 2037 to 2038
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   URL                2 non-null      object 
 1   Player name        2 non-null      object 
 2   Season             2 non-null      object 
 3   Tournament / Team  2 non-null      object 
 4   №                  2 non-null      float64
 5   GP                 2 non-null      float64
 6   G                  2 non-null      float64
 7   Assists            2 non-null      float64
 8   PTS                0 non-null      float64
 9   +/-                0 non-null      float64
 10  +                  0 non-null      float64
 11  -                  0 non-null      float64
 12  PIM                2 non-null      float64
 13  ESG                0 non-null      float64
 14  PPG                0 non-null      float64
 15  SHG                0 non-null      float64
 16  OTG                0 non

Most of the data is missing, and not because it is supposed to be a zero. After all, icetime cannot be zero. Therefore, we need to drop this player from our data.

In [5]:
# Once again, we are using the profile link as primary key due to a possibility of matching names.
players_season = raw_players_season[raw_players_season['URL'] != 'https://en.khl.ru/players/29144/'].copy()

Now we can create a new column indicating whether a player is a skater or a goalie. Let us use the icetime for the separation, since the icetime per game is only tracked for goalies and skaters are supposed to have it as null.

In [6]:
players_season['Position'] = np.where(players_season['TOI/G'].isnull(), 'Goalie', 'Skater')

The icetime data are currently stored in the 'minutes:seconds' format which cannot be used for analysis, so we need to create a separate column containing integer values of icetime in seconds.

We would like to fix the floats in columns where we know the values are supposed to be integers. You cannot score 3.5 goals after all. However, we can have a problem here since converting data to another type requires that there is no NaN values. 

It seems like off-season tournaments are sometimes the ones responsible for messing up our data. If a player have only participated in the off-season tournaments, the rest of his season statistics would end up with NaN values since the off-season statistics is not included. Therefore, let us drop such players from the data despite going through the hassle of including them while scraping the data.

In [7]:
# What off-season tournaments do we have?
players_season['Season'].unique()

array(['Regular season 2014/2015', 'Nadezhda Cup 2013/2014',
       'Regular season 2013/2014', 'Nadezhda Cup 2012/2013',
       'Regular season 2015/2016', 'Regular season 2017/2018',
       'Regular season 2016/2017', 'Playoffs 2010/2011',
       'Regular season 2010/2011', 'Playoffs 2009/2010',
       'Regular season 2009/2010', 'Playoffs 2008/2009',
       'Regular season 2008/2009', 'Regular season 2011/2012',
       'Playoffs 2020/2021', 'Regular season 2020/2021',
       'Playoffs 2019/2020', 'Regular season 2019/2020',
       'Playoffs 2018/2019', 'Regular season 2018/2019',
       'Playoffs 2017/2018', 'Playoffs 2016/2017', 'Playoffs 2015/2016',
       'Playoffs 2011/2012', 'Regular season 2012/2013',
       'Playoffs 2014/2015', 'Playoffs 2012/2013', 'Playoffs 2013/2014'],
      dtype=object)

In [8]:
# Apparently, only the Nadezhda Cup that has run in 2012/2013 and 2013/2014 seasons.
players_season = players_season[~players_season['Season'].isin(['Nadezhda Cup 2012/2013', 'Nadezhda Cup 2013/2014'])]

# We are not interested in players with no games recorded.
players_season = players_season[players_season['GP'] > 0]

# Separating the 'Season' column into the type of season and the years would allow us to more easily sort it.
players_season['Year'] = players_season['Season'].apply(lambda x: x[-9:])
players_season['Season'] = players_season['Season'].apply(lambda x: x[:-10])

# We will do a bit of reordering for the columns.
columns = players_season.columns
players_season = players_season[[col for col in columns[:2]] + ['Position', 'Season', 'Year'] + [col for col in columns[3:-2]]]

# The current column names are not very informative, are they?
header = ['Profile', 'Player', 'Position', 'Season', 'Year', 'Team', 'Number', 'Games', 'Goals', 'Assists', 'Points',
          'Plus_minus', 'Plus', 'Minus', 'Penalties', 'Goals_even', 'Goals_powerplay', 'Goals_shorthanded', 'Goals_overtime',
          'Game_winning_goals', 'Game_winning_shootouts', 'Shots', 'Shots_percentage', 'Shots_game', 'Faceoffs', 'Faceoffs_won',
         'Faceoffs_percentage', 'Icetime_game', 'Shifts_game', 'Hits', 'Shots_blocked', 'Penalties_against', 'Wins',
          'Losses', 'Shootouts', 'Goals_against', 'Saves', 'Saves_percentage', 'Goals_against_average', 'Shutouts', 'Icetime']
players_season.columns = header

# The player's number is currently stored as float, let us change it into object.
players_season['Number'] = players_season['Number'].astype('object')
players_season.head()

,Profile,Player,Position,Season,Year,Team,Number,Games,Goals,Assists,Points,Plus_minus,Plus,Minus,Penalties,Goals_even,Goals_powerplay,Goals_shorthanded,Goals_overtime,Game_winning_goals,Game_winning_shootouts,Shots,Shots_percentage,Shots_game,Faceoffs,Faceoffs_won,Faceoffs_percentage,Icetime_game,Shifts_game,Hits,Shots_blocked,Penalties_against,Wins,Losses,Shootouts,Goals_against,Saves,Saves_percentage,Goals_against_average,Shutouts,Icetime
0,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,Regular season,2014/2015,Amur (Khabarovsk),93.0,13.0,1.0,0.0,1.0,-4.0,1.0,5.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,9.1,0.8,0.0,0.0,-,6:57,9.3,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,Regular season,2013/2014,Amur (Khabarovsk),91.0,12.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,1.2,1.0,0.0,0.0,6:15,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.khl.ru/players/16462/,Maxim Alyapkin,Goalie,Regular season,2015/2016,Torpedo (Nizhny Novgorod Region),31.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,3.0,10.0,76.9,2.98,0.0,60:25
5,https://en.khl.ru/players/16462/,Maxim Alyapkin,Goalie,Regular season,2014/2015,Torpedo (Nizhny Novgorod Region),31.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,2.0,9.0,81.8,3.5,0.0,34:16
6,https://en.khl.ru/players/19200/,Dmitry Ambrozheichik,Skater,Regular season,2017/2018,Dinamo (Minsk),63.0,8.0,0.0,0.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.8,0.0,0.0,-,6:00,8.2,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Can we now change the data from floats to integers? Not really.

Most of our columns still has many NaN values because different statistics are tracked for skaters and goalies. And integers do not like having NaN values in them. It could be worked around but such an approach is not necessarily the best one.

We could, of course, leave it as it is or replace missing values with zeros. However, analysing skaters and goalies together in the future sounds like a bad analysis design since the two groups are very distinct. Therefore, let us separate the data into two distinct dataframes and store skater statistics and goalie statistics separately. That way, we can also change floats into integers within each dataframe separately.

In [9]:
# Thankfully, we have a convenient column to separate on.
skaters_season = players_season[players_season['Position'] == 'Skater'].copy()
goalies_season = players_season[players_season['Position'] == 'Goalie'].copy()

In [10]:
# Most columns are either null or non-null in every row except for 'Hits', 'Shots_blocked' and 'Penalties_against'.
skaters_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17053 entries, 0 to 19673
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Profile                 17053 non-null  object 
 1   Player                  17053 non-null  object 
 2   Position                17053 non-null  object 
 3   Season                  17053 non-null  object 
 4   Year                    17053 non-null  object 
 5   Team                    17053 non-null  object 
 6   Number                  16126 non-null  object 
 7   Games                   17053 non-null  float64
 8   Goals                   17053 non-null  float64
 9   Assists                 17053 non-null  float64
 10  Points                  17053 non-null  float64
 11  Plus_minus              17053 non-null  float64
 12  Plus                    17053 non-null  float64
 13  Minus                   17053 non-null  float64
 14  Penalties               17053 non-null

In [11]:
# And all is perfect here!
goalies_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 4 to 19650
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Profile                 1475 non-null   object 
 1   Player                  1475 non-null   object 
 2   Position                1475 non-null   object 
 3   Season                  1475 non-null   object 
 4   Year                    1475 non-null   object 
 5   Team                    1475 non-null   object 
 6   Number                  1405 non-null   object 
 7   Games                   1475 non-null   float64
 8   Goals                   1475 non-null   float64
 9   Assists                 1475 non-null   float64
 10  Points                  0 non-null      float64
 11  Plus_minus              0 non-null      float64
 12  Plus                    0 non-null      float64
 13  Minus                   0 non-null      float64
 14  Penalties               1475 non-null  

What is going on here? It feels as if for some reason those three columns are only recorded part of the time. Is it happening all the time?

In [12]:
# Distribution of NaN values for 'Hits' by season.
skaters_season[skaters_season['Hits'].isnull()].groupby(['Season', 'Year']).size()

Season          Year     
Playoffs        2008/2009     338
                2009/2010     352
                2010/2011     357
                2011/2012     366
                2012/2013     374
                2013/2014     373
Regular season  2008/2009     833
                2009/2010     795
                2010/2011     824
                2011/2012     808
                2012/2013     967
                2013/2014    1019
dtype: int64

Would you look at that, the missing values are only the case for seasons 2008/2009 through 2013/2014! Clearly, the indicators have just not been tracked in though years. Mystery solved.

However, what should we do with it? Replacing it with zeros would not be very fair and can mess up our analysis. At the same time, we cannot change the column to integers without replacing the NaN values. Oh well, we might leave it as is for now and keep in mind that any further analysis needs to take into account that change from season 2014/2015 onwards.

Meanwhile, two columns that require special attention would be 'Icetime_game' and 'Icetime' which are stored in the format 'minutes:seconds' and are thus not convertible to floats. A new column will be added for both of them, calculated as an integer value in seconds.

In [13]:
def icetime_seconds(icetime):
    time_list = icetime.split(':')
    minutes = int(time_list[0])
    seconds = int(time_list[1])
    return minutes * 60 + seconds

Finally we can remove the null columns and change the non-null ones to integers. Actually, in a few cases we would need to change the columns to floats for things such as '%SOG' (percentage of shots on goal that scored) which seem to be stored as objects right now.

At the same time, some columns that we want to store as floats have '-' for their value, which cannot be converted into a float value. This is because they are obtained by dividing one statistics by another and one of the two may not be suitable for such operation. We are going to replace those values with NaN.

But before we do all that, let us also drop the rows with zero icetime per game (skaters) or zero icetime (goalies). After all, we are not really interested in players who have not essentially participated in any matches.

In [14]:
# We can copy paste parts of the previously created list of column names instead of typing them up manually.
print(header)

['Profile', 'Player', 'Position', 'Season', 'Year', 'Team', 'Number', 'Games', 'Goals', 'Assists', 'Points', 'Plus_minus', 'Plus', 'Minus', 'Penalties', 'Goals_even', 'Goals_powerplay', 'Goals_shorthanded', 'Goals_overtime', 'Game_winning_goals', 'Game_winning_shootouts', 'Shots', 'Shots_percentage', 'Shots_game', 'Faceoffs', 'Faceoffs_won', 'Faceoffs_percentage', 'Icetime_game', 'Shifts_game', 'Hits', 'Shots_blocked', 'Penalties_against', 'Wins', 'Losses', 'Shootouts', 'Goals_against', 'Saves', 'Saves_percentage', 'Goals_against_average', 'Shutouts', 'Icetime']


In [15]:
# What values do we have in case of zero icetime per game?
skaters_season.groupby('Icetime_game').size().head()

Icetime_game
-       8
0:04    1
0:05    2
0:06    2
0:08    1
dtype: int64

In [16]:
# Starting with the more numerous skaters.
skaters_season = skaters_season[skaters_season['Icetime_game'] != '-']
skaters_season.drop(['Wins', 'Losses', 'Shootouts', 'Goals_against', 'Saves', 'Saves_percentage',
                     'Goals_against_average','Shutouts', 'Icetime'], axis=1, inplace=True)

# Columns to be changed into integers.
skaters_int = ['Games', 'Goals', 'Assists', 'Points', 'Plus_minus', 'Plus', 'Minus', 'Penalties', 'Goals_even',
               'Goals_powerplay', 'Goals_shorthanded', 'Goals_overtime', 'Game_winning_goals', 'Game_winning_shootouts',
               'Shots', 'Faceoffs', 'Faceoffs_won']
skaters_season[skaters_int] = skaters_season[skaters_int].astype('int')

# Columns to be changed into floats.
skaters_float = ['Shots_percentage', 'Shots_game', 'Faceoffs_percentage', 'Shifts_game']
skaters_season[skaters_float] = skaters_season[skaters_float].replace('-', np.NaN).astype('float')

# Finally, let us add the icetime in seconds and move it to be right after our existing icetime.
skaters_season['Icetime_game_seconds'] = skaters_season['Icetime_game'].apply(icetime_seconds)
header_skaters = skaters_season.columns
skaters_season = skaters_season[[col for col in header_skaters[:28]] + ['Icetime_game_seconds'] +
                                [col for col in header_skaters[28:-1]]]
skaters_season.head()

,Profile,Player,Position,Season,Year,Team,Number,Games,Goals,Assists,Points,Plus_minus,Plus,Minus,Penalties,Goals_even,Goals_powerplay,Goals_shorthanded,Goals_overtime,Game_winning_goals,Game_winning_shootouts,Shots,Shots_percentage,Shots_game,Faceoffs,Faceoffs_won,Faceoffs_percentage,Icetime_game,Icetime_game_seconds,Shifts_game,Hits,Shots_blocked,Penalties_against
0,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,Regular season,2014/2015,Amur (Khabarovsk),93.0,13,1,0,1,-4,1,5,6,1,0,0,0,0,0,11,9.1,0.8,0,0,NaN,6:57,417,9.3,1.0,2.0,1.0
2,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,Regular season,2013/2014,Amur (Khabarovsk),91.0,12,0,0,0,0,1,1,0,0,0,0,0,0,0,14,0.0,1.2,1,0,0.0,6:15,375,8.0,NaN,NaN,NaN
6,https://en.khl.ru/players/19200/,Dmitry Ambrozheichik,Skater,Regular season,2017/2018,Dinamo (Minsk),63.0,8,0,0,0,-1,1,2,0,0,0,0,0,0,0,6,0.0,0.8,0,0,NaN,6:00,360,8.2,2.0,2.0,0.0
7,https://en.khl.ru/players/19200/,Dmitry Ambrozheichik,Skater,Regular season,2016/2017,Dinamo (Minsk),15.0,20,3,1,4,1,5,4,10,3,0,0,0,0,0,21,14.3,1.1,7,3,42.9,9:43,583,12.7,10.0,7.0,9.0
8,https://en.khl.ru/players/19200/,Dmitry Ambrozheichik,Skater,Regular season,2015/2016,Dinamo (Minsk),24.0,11,1,0,1,1,3,2,0,1,0,0,0,0,0,5,20.0,0.5,0,0,NaN,4:43,283,8.5,1.0,0.0,1.0


In [17]:
# What values do we have in case of zero total icetime?
goalies_season.groupby('Icetime').size().head()

Icetime
-       2
0:10    1
0:13    1
0:18    1
0:45    1
dtype: int64

In [18]:
# Now for the goalies.
goalies_season = goalies_season[goalies_season['Icetime'] != '-']
goalies_season.drop(['Points', 'Plus_minus', 'Plus', 'Minus', 'Goals_even', 'Goals_powerplay', 'Goals_shorthanded',
                     'Goals_overtime', 'Game_winning_goals', 'Game_winning_shootouts', 'Shots_percentage', 'Shots_game',
                     'Faceoffs', 'Faceoffs_won', 'Faceoffs_percentage', 'Icetime_game','Shifts_game', 'Hits',
                     'Shots_blocked', 'Penalties_against'], axis=1, inplace=True)

# Columns to be changed into integers.
goalies_int = ['Games', 'Goals', 'Assists', 'Penalties', 'Shots', 'Wins', 'Losses', 'Shootouts', 'Goals_against',
               'Saves', 'Shutouts']
goalies_season[goalies_int] = goalies_season[goalies_int].astype('int')

# Columns to be changed into floats.
goalies_float = ['Saves_percentage', 'Goals_against_average']
goalies_season[goalies_float] = goalies_season[goalies_float].replace('-', np.NaN).astype('float')

# At least we will not have to move the icetime in seconds now, since icetime is already at the end of the dataframe.
goalies_season['Icetime_seconds'] = goalies_season['Icetime'].apply(icetime_seconds)
goalies_season.head()

,Profile,Player,Position,Season,Year,Team,Number,Games,Goals,Assists,Penalties,Shots,Wins,Losses,Shootouts,Goals_against,Saves,Saves_percentage,Goals_against_average,Shutouts,Icetime,Icetime_seconds
4,https://en.khl.ru/players/16462/,Maxim Alyapkin,Goalie,Regular season,2015/2016,Torpedo (Nizhny Novgorod Region),31.0,2,0,0,0,13,1,1,0,3,10,76.9,2.98,0,60:25,3625
5,https://en.khl.ru/players/16462/,Maxim Alyapkin,Goalie,Regular season,2014/2015,Torpedo (Nizhny Novgorod Region),31.0,1,0,0,0,11,0,1,0,2,9,81.8,3.50,0,34:16,2056
17,https://en.khl.ru/players/16898/,Artyom Artemyev,Goalie,Regular season,2015/2016,Sochi (Sochi),31.0,1,0,0,0,12,0,0,0,3,9,75.0,3.36,0,53:38,3218
18,https://en.khl.ru/players/16898/,Artyom Artemyev,Goalie,Regular season,2014/2015,Atlant (Moscow Region),31.0,5,0,0,0,112,1,2,1,9,103,92.0,2.07,0,261:14,15674
19,https://en.khl.ru/players/16898/,Artyom Artemyev,Goalie,Regular season,2013/2014,Severstal (Cherepovets),52.0,2,0,0,0,16,0,0,0,3,13,81.2,8.89,0,20:15,1215


Everything seems to be in order, good job us! Now, for the best part.

In [19]:
skaters_season.to_csv('../data/skaters_season.csv', encoding='utf8', index=False)
goalies_season.to_csv('../data/goalies_season.csv', encoding='utf8', index=False)